In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

# 加载模型
model = models.resnet50(weights=None)  # 这里不加载预训练权重
model.fc = torch.nn.Linear(model.fc.in_features, 7)  # 再次定义适合FER-2013的全连接层

# 2. 加载本地保存的模型权重
model_path = '/hy-tmp/finetuned_model/FER_5epochs.pth'
model.load_state_dict(torch.load(model_path, weights_only=True))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()  # 设置为评估模式


In [ ]:
from PIL import Image
import torch.nn.functional as F

# 定义与训练时相同的预处理方式
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 使用 ImageNet 均值和标准差进行归一化
])


In [ ]:
#单张图片推理

In [ ]:

# 加载一张图片
image_path = './data/test/angry/PrivateTest_10131363.jpg'  # 替换为你图片的路径
image = Image.open(image_path)


# 对图片进行预处理
input_image = transform(image).unsqueeze(0)  # 扩展维度，变成[1, C, H, W]（batch size为1）
input_image = input_image.to(device)

with torch.no_grad():  # 关闭梯度计算，节省内存
    output = model(input_image)

probabilities = F.softmax(output, dim=1)

# 获取每个类别的概率和对应的标签
# 假设 emotion_labels 是一个字典，将类别的索引映射到情绪标签
emotion_labels = {0: "angry", 1: "disgust", 2: "fear", 3: "happy", 4: "neutral", 5: "sad", 6: "surprise"}

# 对每个样本获取概率值和对应标签，并按概率降序排列
for i in range(probabilities.size(0)):  # 遍历每一个样本（假设batch_size > 1）
    sample_probs = probabilities[i]  # 获取每个样本的概率分布
    prob_with_labels = [(emotion_labels[j], sample_probs[j].item()) for j in range(len(emotion_labels))]
    
    # 按概率从高到低排序
    sorted_probabilities = sorted(prob_with_labels, key=lambda x: x[1], reverse=True)

    print(f"Sample {i+1}:")
    for emotion, prob in sorted_probabilities:
        print(f"  {emotion}: {prob:.4f}")
    print()


# 如果是分类任务，使用 `torch.max` 得到最大概率的类别
##_, predicted = torch.max(output, 1)

# 打印预测结果
##print(f"Predicted class: {predicted.item()}")



In [ ]:
#多张图片推理

In [ ]:
# 定义与训练时相同的预处理方式
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 使用 ImageNet 均值和标准差进行归一化
])

test_dir = "./data/test"
test_data = ImageFolder(test_dir, transform=test_transforms)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, pin_memory=True)


